1. Загрузить файл длиной не менее 2000 символов.

In [ ]:
# open file, read file, get length, print length, return file content
def load_file(filepath: str):
    with open(filepath, 'r') as f:
        file_content = f.read()
        symbols_count = len(file_content)
        print(f'File contains {symbols_count} symbols')
    
    return file_content

In [20]:
filepath = '../DZ_1/Q&A Lazko Viktoria.txt'
file_text = load_file(filepath)


File contains 2121 symbols


2. Составить программу, которая считает число уникальных слов в тексте (без критерия
схожести)

In [18]:
# split text to words by space, make set to get unique words, get length of words set, return count
def calc_unique_words(text: str):
    words = text.split()
    uniques_count = len(set(words))
    print(f'There are {uniques_count} unique words in text')
    
    return uniques_count


In [21]:
uniques_count = calc_unique_words(file_text)


There are 190 unique words in text


3. Составить программу, которая считает число гласных и согласных букв.